# Sabermetrics

Class project to learn about ensembling and combining models to find a good baseball player within budget. The goal is to predict IP and RA9 for the 2013 season, Moneyball-style...although I never got around to watching the movie. Can't take credit for the starter code, can take credit for about half of the improvements.

## Import

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Read in the data

The data set here is a data set of pitcher information combined with predictions on the IP (or _innings pitched_) and RA9 (or _runs allowed per 9 innings_). These predictions come from four different groups as stated in the case.

`train_saber.csv` is predictions for the 2012 season, which has already happened, so we also have the realizations for IP and RA9 for the 2012 season. We are trying to predict the 2013 season. `test_saber.csv` has all of the currently available information that we would have in making his hiring decisions, but it does not have any data from the 2013 season.

In [2]:
train_df = pd.read_csv("train_saber.csv")

In [3]:
display(train_df.head())

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate,IP,RA9
0,Aaron Crow,127,4.187402,56,4.132736,150,3.607945,67.3,4.963657,crowaa01,...,R,R,3/31/2011,9/28/2014,crowa001,crowaa01,NaN,1986-11-10,64.666667,3.757732
1,Aaron Harang,164,4.602863,156,4.701261,180,4.646595,133.7,4.712194,haranaa01,...,R,R,5/25/2002,9/27/2014,haraa001,haranaa01,NaN,1978-05-09,179.666667,4.257885
2,Adam Wainwright,196,3.443947,83,3.203417,175,3.771942,173.3,3.411148,wainwad01,...,R,R,9/11/2005,9/22/2014,waina001,wainwad01,NaN,1981-08-30,198.666667,4.348993
3,Alex Cobb,103,4.099937,86,4.176469,110,4.209269,122.0,4.351400,cobbal01,...,R,R,5/1/2011,9/28/2014,cobba001,cobbal01,NaN,1987-10-07,136.333333,4.422983
4,Alexi Ogando,175,3.826608,130,3.826608,185,4.045271,150.0,4.067137,ogandal01,...,R,R,6/15/2010,6/3/2014,ogana001,ogandal01,NaN,1983-10-05,66.000000,3.545455


In [4]:
test_df = pd.read_csv("test_saber.csv")

In [5]:
display(test_df.head())

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate,IP,RA9
0,Aaron Crow,66,3.538949,63.7,3.843843,60,3.593394,64.0,4.137848,crowaa01,...,R,R,2011-03-31,NaN,crowa001,crowaa01,NaN,1986-11-10,NaN,NaN
1,Adam Wainwright,215,3.462725,159.3,3.996290,195,3.462725,173.7,3.724063,wainwad01,...,R,R,2005-09-11,NaN,waina001,wainwad01,NaN,1981-08-30,NaN,NaN
2,Addison Reed,61,3.789397,53.3,4.595189,60,4.246739,66.7,4.257628,reedad01,...,L,R,2011-09-04,NaN,reeda001,reedad01,NaN,1988-12-27,NaN,NaN
3,Al Alburquerque,62,2.929161,36.0,3.538949,55,3.027162,34.7,4.235849,albural01,...,R,R,2011-04-15,NaN,albua001,albural01,NaN,1986-06-10,NaN,NaN
4,Alex Cobb,164,4.159626,133.7,4.181404,140,3.920066,149.3,4.529854,cobbal01,...,R,R,2011-05-01,NaN,cobba001,cobbal01,NaN,1987-10-07,NaN,NaN


In [6]:
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.concat([pd.DataFrame(df.columns, columns=['Variable Name']), 
                      pd.DataFrame(df.dtypes.values.reshape([-1,1]), columns=['Data Type']),
                      pd.DataFrame(df.isnull().sum().values, columns=['Missing Values']), 
                      pd.DataFrame([df[name].nunique() for name in df.columns], columns=['Unique Values'])], 
                     axis=1).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [7]:
summarize_dataframe(train_df)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
NAME,object,0,182,182.0,182,Aaron Crow,1,,,,,,,
IP_fans,int64,0,107,182.0,,,,141.467033,58.286836,48.0,69.0,159.5,191.5,242.0
RA9_fans,float64,0,120,182.0,,,,3.947053,0.647344,2.536494,3.44668,3.974206,4.479864,5.444716
IP_marcel,int64,0,100,182.0,,,,119.445055,51.185879,37.0,64.0,126.5,167.0,208.0
RA9_marcel,float64,0,115,182.0,,,,4.058247,0.534799,2.897289,3.673544,4.001538,4.468931,5.499382
IP_rotochamp,int64,0,28,182.0,,,,136.236264,58.484396,30.0,60.0,155.0,190.0,220.0
RA9_rotochamp,float64,0,107,182.0,,,,4.008146,0.740808,1.825839,3.545079,4.045271,4.608329,5.783644
IP_zips,float64,0,148,182.0,,,,128.71978,53.35577,41.7,70.325,140.0,173.225,237.3
RA9_zips,float64,0,129,182.0,,,,4.192568,0.729662,2.405296,3.698143,4.14367,4.695794,6.341236
playerID,object,0,182,182.0,182,crowaa01,1,,,,,,,


In [8]:
summarize_dataframe(test_df)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
NAME,object,0,191,191.0,191,Aaron Crow,1,,,,,,,
IP_fans,int64,0,99,191.0,,,,139.125654,57.950931,51.0,70.0,158.0,191.0,233.0
RA9_fans,float64,0,120,191.0,,,,3.894982,0.63256,2.450041,3.440947,3.952734,4.371963,5.128753
IP_marcel,float64,0,150,191.0,,,,116.719372,51.228593,34.0,65.0,120.0,165.2,204.3
RA9_marcel,float64,0,117,191.0,,,,4.080666,0.514831,2.754935,3.702285,4.105181,4.453631,5.477204
IP_rotochamp,int64,0,33,191.0,,,,130.026178,59.616091,25.0,60.0,150.0,185.0,220.0
RA9_rotochamp,float64,0,116,191.0,,,,3.979244,0.69107,1.676917,3.52806,4.028957,4.475409,5.488093
IP_zips,float64,0,161,191.0,,,,128.963351,53.051221,23.0,68.65,141.3,169.85,228.3
RA9_zips,float64,0,139,191.0,,,,4.25415,0.754846,1.709584,3.734952,4.235849,4.769414,6.184994
playerID,object,0,191,191.0,191,crowaa01,1,,,,,,,


## Model building

In [9]:
def accuracy(y_true, y_pred):
    """Function that returns a table showing RMSE and MAE."""
    acc_df = pd.DataFrame(data = {"RMSE": [mean_squared_error(y_true, y_pred)**(1/2)],
                                  "MAE": [mean_absolute_error(y_true, y_pred)],
                                  "R^2": [r2_score(y_true, y_pred)]})
    display(acc_df.style.hide_index())

## Building Ensembles

In [10]:
import statsmodels.formula.api as smf

In [17]:
lm_RA9 = smf.ols(formula='RA9 ~ RA9_fans+RA9_marcel+RA9_rotochamp+RA9_zips', data=train_df).fit()
lm_RA9.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RA9   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     7.731
Date:                Fri, 24 Dec 2021   Prob (F-statistic):           9.17e-06
Time:                        19:22:54   Log-Likelihood:                -322.68
No. Observations:                 182   AIC:                             655.4
Df Residuals:                     177   BIC:                             671.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         1.0520      0.845      1.245      0.215      -0.615       2.719
RA9_fans          0.4409      0.432      1.020      0.309      -0.412       1.294
RA9_marcel       -0.0496      0.420     -0.118      0.906      -0.879       0.780
RA9_rotochamp     0.6758      0.309      2.186      0.030       0.066       1.286
RA9_zips         -0.2252      0.385     -0.585      0.559      -0.985       0.535
==============================================================================
Omnibus:                       76.887   Durbin-Watson:                   1.729
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              308.669
Skew:                           1.621   Prob(JB):                     9.40e-68
Kurtosis:                       8.494   Cond. No.                         69.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
RA9_pred_2012 = train_df["RA9_fans"]*0.4409 \
+train_df["RA9_marcel"]*-0.0496 \
+train_df["RA9_rotochamp"]*0.6758 \
+train_df["RA9_zips"]*-0.22528 + 1.0520

In [20]:
accuracy(train_df["RA9"], RA9_pred_2012)

RMSE,MAE,R^2
1.424835,1.021027,0.148729


In [21]:
lm_IP = smf.ols(formula='IP ~ IP_fans+IP_marcel+IP_rotochamp+IP_zips', data=train_df).fit()
lm_IP.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     IP   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     39.31
Date:                Fri, 24 Dec 2021   Prob (F-statistic):           1.57e-23
Time:                        19:25:28   Log-Likelihood:                -968.76
No. Observations:                 182   AIC:                             1948.
Df Residuals:                     177   BIC:                             1964.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.2621      9.983     -0.227      0.821     -21.964      17.439
IP_fans          0.5642      0.258      2.189      0.030       0.056       1.073
IP_marcel        0.0590      0.188      0.314      0.754      -0.312       0.430
IP_rotochamp     0.0291      0.234      0.125      0.901      -0.433       0.491
IP_zips          0.1908      0.195      0.979      0.329      -0.194       0.575
==============================================================================
Omnibus:                       13.514   Durbin-Watson:                   1.840
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.680
Skew:                          -0.693   Prob(JB):                     0.000649
Kurtosis:                       3.132   Cond. No.                         762.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
IP_pred_2012 = train_df["IP_fans"]*0.5642 \
+train_df["IP_marcel"]*0.0590 \
+train_df["IP_rotochamp"]*0.0291 \
+train_df["IP_zips"]*0.1908 -2.2621

In [23]:
accuracy(train_df["IP"], IP_pred_2012)

RMSE,MAE,R^2
49.597127,38.914154,0.470434


# Pick your Pitchers

After you build your predictions, use them, along with the other data in the case, to choose the pitchers you will sign. You will need to get the predicted IP and RA9 of all of the free agent pitchers in exhibit 4. Below is an easy way to find the indices of all of the pitchers.

In [24]:
IP_pred_2013 = test_df["IP_fans"]*0.5642 \
+test_df["IP_marcel"]*0.0590 \
+test_df["IP_rotochamp"]*0.0291 \
+test_df["IP_zips"]*0.1908 -2.2621

In [25]:
print(IP_pred_2013)

0       52.69060
1      167.25606
2       49.77116
3       43.06356
4      130.71544
         ...    
186    166.90340
187    169.74320
188     93.87240
189    130.46796
190    171.10364
Length: 191, dtype: float64


In [26]:
RA9_pred_2013 = test_df["RA9_fans"]*0.4409 \
+test_df["RA9_marcel"]*-0.0496 \
+test_df["RA9_rotochamp"]*0.6758 \
+test_df["RA9_zips"]*-0.22528 + 1.0520

In [27]:
free_agents = ["Andy Pettitte",
               "Bartolo Colon",
               "Brandon McCarthy",
               "Francisco Liriano",
               "Hiroki Kuroda",
               "Joe Blanton",
               "Joe Saunders",
               "Kevin Correia",
               "Paul Maholm",
               "Ryan Dempster",
               "Shaun Marcum",
               "Anibal Sanchez",
               "Brett Myers",
               "Hisashi Iwakuma",
               "Jeremy Guthrie",
               "Zack Greinke"]

In [28]:
test_df.loc[test_df["NAME"].isin(free_agents)]

,NAME,IP_fans,RA9_fans,IP_marcel,RA9_marcel,IP_rotochamp,RA9_rotochamp,IP_zips,RA9_zips,playerID,...,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate,IP,RA9
9,Andy Pettitte,145,4.018068,97.7,3.909177,140,3.571616,90.3,4.453631,pettian01,...,L,L,1995-04-29,NaN,petta001,pettian01,NaN,1972-06-15,NaN,NaN
10,Anibal Sanchez,197,4.039846,177.7,4.137848,200,3.974512,185.0,4.606078,sanchan01,...,R,R,2006-06-25,NaN,sanca004,sanchan01,NaN,1984-02-27,NaN,NaN
14,Bartolo Colon,135,4.322962,151.0,4.214071,120,4.410075,127.0,4.475409,colonba01,...,R,R,1997-04-04,NaN,colob001,colonba01,NaN,1973-05-24,NaN,NaN
21,Brandon McCarthy,158,3.680507,132.7,3.843843,130,4.301184,118.7,4.214071,mccarbr01,...,R,R,2005-05-22,NaN,mccab001,mccarbr01,NaN,1983-07-07,NaN,NaN
25,Brett Myers,178,4.518965,90.3,4.344740,150,4.181404,178.0,4.627856,myersbr01,...,R,R,2002-07-24,NaN,myerb001,myersbr01,NaN,1980-08-17,NaN,NaN
63,Francisco Liriano,163,4.595189,147.7,5.106975,160,5.019863,154.7,4.050735,liriafr01,...,L,L,2005-09-05,NaN,lirif001,liriafr01,NaN,1983-10-26,NaN,NaN
73,Hiroki Kuroda,198,3.952734,190.3,3.811176,180,3.647839,186.0,4.638745,kurodhi01,...,R,R,2008-04-04,NaN,kuroh001,kurodhi01,NaN,1975-02-10,NaN,NaN
74,Hisashi Iwakuma,179,4.028957,106.3,3.865621,160,3.985401,128.3,4.203182,iwakuhi01,...,R,R,2012-04-20,NaN,iwakh001,iwakuhi01,NaN,1981-04-12,NaN,NaN
92,Jeremy Guthrie,196,4.704080,169.0,4.812970,195,4.769414,163.7,4.791192,guthrje01,...,R,R,2004-08-28,NaN,guthj001,guthrje01,NaN,1979-04-08,NaN,NaN
98,Joe Blanton,172,4.943639,157.3,5.041641,170,4.497187,130.3,4.889194,blantjo01,...,R,R,2004-09-21,NaN,blanj001,blantjo01,NaN,1980-12-11,NaN,NaN


In [33]:
free_agent_indices = test_df["NAME"].isin(free_agents)

In [34]:
free_agent_indices

0      False
1      False
2      False
3      False
4      False
       ...  
186    False
187    False
188    False
189    False
190     True
Name: NAME, Length: 191, dtype: bool

The following code will help you more easily read out your predictions for the individual pitchers in order to use the predictions in your decision. Note that until you have the variables `IP_pred_2013` and `RA9_pred_2013`, you will get an error if you try to run the below cell.

In [35]:
pd.DataFrame({'Name': test_df[free_agent_indices]["NAME"], 
              'Prediction IP 2013': IP_pred_2013[free_agent_indices], 
              'Prediction RA9 2013': RA9_pred_2013[free_agent_indices]}).set_index('Name')

,Prediction IP 2013,Prediction RA9 2013
Name,,
Andy Pettitte,106.61444,4.040055
Anibal Sanchez,160.48760,4.276249
Bartolo Colon,110.53750,4.721084
Brandon McCarthy,121.14176,4.441475
Brett Myers,141.82060,4.612142
Francisco Liriano,132.58956,5.304586
Hiroki Kuroda,161.40400,4.025919
Hisashi Iwakuma,134.13704,4.383073
Jeremy Guthrie,155.20056,5.031116


# Write Out Predictions

In [36]:
predictions_df = pd.concat([test_df['NAME'], IP_pred_2013.rename('IP')], axis=1)

In [37]:
predictions_df.to_csv("IP_predictions_2013.csv", index=False)

In class, you will receive a realizations data set. Assuming that you have `IP_pred_2013` and `RA9_pred_2013` calculated from work above, running the below code will show your accuracy.

In [77]:
test_real_df = pd.read_csv("../Shared Data (Read Only)/Sabermetrics_SPF/test_saber_realization.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../Shared Data (Read Only)/Sabermetrics_SPF/test_saber_realization.csv'

In [ ]:
print("Your IP prediction performance was:")
accuracy(test_real_df["IP"], IP_pred_2013)

In [ ]:
print("Your RA9 prediction performance was:")
accuracy(test_real_df["RA9"], RA9_pred_2013)